In [10]:
import numpy as np
import pandas as pd
from pymongo import *
from datetime import time, tzinfo, timedelta, datetime

client = MongoClient('52.79.239.183', 27017)
if client:
    print("DB connection complete!!")

def Get_each_Coin_DF(coinName):
    
    DB_Coin = client[coinName]
    Collection = DB_Coin['BTC/USD_30MIN']
    DB_schema = []
    for collect in Collection.find():
        temp_record = {}
        temp_record["%s_open" % coinName] = collect['price_open']
        temp_record["%s_high" % coinName] = collect['price_high']
        temp_record["%s_low" % coinName] = collect['price_low']
        temp_record["%s_close" % coinName] = collect['price_close']
        
        temp_record['Date'] = collect['time_period_end']
        temp_record['label'] = coinName
        temp_record["%s_vol" % coinName] = collect['volume_traded']
        DB_schema.append(temp_record)
        del temp_record

    DF = pd.DataFrame(DB_schema)
    DF['Date'] = DF['Date'].apply(pd.to_datetime, errors='coerce')
    DF.index = DF['Date']
    DF = DF.sort_values(by='Date')
    DF = DF[["%s_open" % coinName, "%s_high" % coinName, "%s_low" % coinName, "%s_close" % coinName, "%s_vol" % coinName]] 
    return DF

def Get_Mean_Median():
    BinanceDF = Get_each_Coin_DF("BINANCE")
    HuobiDF = Get_each_Coin_DF("HUOBI")
    OkexDF = Get_each_Coin_DF("OKEX")
    HitbtcDF = Get_each_Coin_DF("HITBTC")
    BitfinexDF = Get_each_Coin_DF("BITFINEX")
    BitstampDF = Get_each_Coin_DF("BITSTAMP")
    CoinbaseDF = Get_each_Coin_DF("COINBASE")
    CryptopiaDF = Get_each_Coin_DF("CRYPTOPIA")
    PoloniexDF = Get_each_Coin_DF("POLONIEX")
    BitmexDF = Get_each_Coin_DF("BITMEX")
    
    JoinedDF = pd.merge(BinanceDF, HuobiDF, how='outer', left_index=True, right_index=True)
    for df in [OkexDF, HitbtcDF, BitfinexDF, BitstampDF, CoinbaseDF, CryptopiaDF, PoloniexDF, BitmexDF]:
        JoinedDF = pd.merge(JoinedDF, df, how='outer', left_index=True, right_index=True)
    
    Top10 = ['BINANCE', 'HUOBI', 'OKEX', 'HITBTC', 'BITFINEX', 'BITSTAMP', 'COINBASE', 'CRYPTOPIA', 'POLONIEX', 'BITMEX']
    VolDF = JoinedDF[[Name + "_vol" for Name in Top10]]
    CloseDF = JoinedDF[[Name + "_close" for Name in Top10]]
    OpenDF = JoinedDF[[Name + "_open" for Name in Top10]]
    LowDF = JoinedDF[[Name + "_low" for Name in Top10]]
    HighDF = JoinedDF[[Name + "_high" for Name in Top10]]
    MeanMedian = [OpenDF.mean(1), HighDF.mean(1), LowDF.mean(1), CloseDF.mean(1), VolDF.median(1)]
    DF = pd.concat(MeanMedian, axis=1)
    DF.columns = ['open', 'high', 'low', 'close', 'volume']
    #DF.to_csv("./mean_price.csv", sep=",")
    
    return JoinedDF

Get_Mean_Median()

DB connection complete!!


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: 'Date' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


,BINANCE_open,BINANCE_high,BINANCE_low,BINANCE_close,BINANCE_vol,HUOBI_open,HUOBI_high,HUOBI_low,HUOBI_close,HUOBI_vol,...,POLONIEX_open,POLONIEX_high,POLONIEX_low,POLONIEX_close,POLONIEX_vol,BITMEX_open,BITMEX_high,BITMEX_low,BITMEX_close,BITMEX_vol
Date,,,,,,,,,,,,,,,,,,,,,
2017-08-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2855.000000,2857.502459,2821.000000,2847.798299,289.959654,NaN,NaN,NaN,NaN,NaN
2017-08-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2847.798300,2851.231241,2820.000000,2820.000000,254.646706,NaN,NaN,NaN,NaN,NaN
2017-08-01 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2820.000000,2847.114067,2816.675887,2844.231438,143.676806,NaN,NaN,NaN,NaN,NaN
2017-08-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2843.366414,2859.180851,2840.600000,2856.259232,155.582851,NaN,NaN,NaN,NaN,NaN
2017-08-01 02:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2856.987091,2857.272818,2843.559625,2851.000000,98.476503,NaN,NaN,NaN,NaN,NaN
2017-08-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2850.734131,2855.715788,2844.462208,2855.715788,153.405238,NaN,NaN,NaN,NaN,NaN
2017-08-01 03:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2855.715788,2856.560377,2844.231438,2852.000000,110.084341,NaN,NaN,NaN,NaN,NaN
2017-08-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2852.000000,2867.445750,2845.000000,2866.445750,200.603357,NaN,NaN,NaN,NaN,NaN
2017-08-01 04:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2866.445750,2871.648020,2856.679821,2866.688152,240.433236,NaN,NaN,NaN,NaN,NaN
